# pytorchのモデル可視化方法
- はっきり言って乱立している
- 以下を紹介
    - torchinfo
    - 

## torchinfo
- jupyterで途中確認したくなった時に気楽に確認可能
- 保存はしてくれない。テキストベースなので見にくい。

In [3]:
import torch
from torchvision.models import resnet18
from torchinfo import summary


model = resnet18()
batch_size = 2

summary(
    model,
    input_size=(batch_size, 3, 224, 224),
    col_names=["output_size", "num_params"],
)

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   --                        --
├─Conv2d: 1-1                            [2, 64, 112, 112]         9,408
├─BatchNorm2d: 1-2                       [2, 64, 112, 112]         128
├─ReLU: 1-3                              [2, 64, 112, 112]         --
├─MaxPool2d: 1-4                         [2, 64, 56, 56]           --
├─Sequential: 1-5                        [2, 64, 56, 56]           --
│    └─BasicBlock: 2-1                   [2, 64, 56, 56]           --
│    │    └─Conv2d: 3-1                  [2, 64, 56, 56]           36,864
│    │    └─BatchNorm2d: 3-2             [2, 64, 56, 56]           128
│    │    └─ReLU: 3-3                    [2, 64, 56, 56]           --
│    │    └─Conv2d: 3-4                  [2, 64, 56, 56]           36,864
│    │    └─BatchNorm2d: 3-5             [2, 64, 56, 56]           128
│    │    └─ReLU: 3-6                    [2, 64, 56, 56]           --
│

## tensorboardでの確認
- 簡単、見やすい、汎用的
- tensorboardが必要なのでだるい、使い方よくわからん
- そのうちtensorboardの使い方についても書くかも
- dockerで`docker run --runtime=nvidia -it --rm -v $(realpath ~/notebooks):/tf/notebooks -p 8888:8888 -p 6006:6006 tensorflow/tensorflow:1.12.0-gpu-py3`（このコマンドがあってるかは保証しない）
- とするか、googlecolabで確認するかのどちらかが良い
- 参考1: https://qiita.com/nj_ryoo0/items/f3aac1c0e92b3295c101
- 参考2: https://www.msdd.info/entry/2019/12/22/080000

In [6]:
import numpy as np
from torch.utils.tensorboard import SummaryWriter


# ログをとる対象を増やしてみる
np.random.seed(111)
x1 = np.random.randn(100)
y1 = x1.cumsum()

x2 = np.random.randn(100)
y2 = x2.cumsum()

writer = SummaryWriter(log_dir="./logs/exp1") # exp1のように分けると、比較の時に楽

# tag = "group_name/value_name" の形式にすることで、別々に見てくれる
for i in range(100):
    writer.add_scalar("X/x1", x1[i], i)
    writer.add_scalar("Y/y1", y1[i], i)
    writer.add_scalar("X/x2", x2[i], i)
    writer.add_scalar("Y/y2", y2[i], i)

writer.close()


- 上を実行するとログが作られる

```tensorboard --logdir logのディレクトリ```で閲覧可能

- pytorchのネットワークに対しては以下のようにする

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TestModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1=nn.Linear(10,5)
        self.fc2=nn.Linear(5,2)
        

    def forward(self,x):
        x=F.relu(self.fc1(x))
        x=F.softmax(self.fc2(x))
        return x

model=TestModel()
print(model)

TestModel(
  (fc1): Linear(in_features=10, out_features=5, bias=True)
  (fc2): Linear(in_features=5, out_features=2, bias=True)
)


In [8]:
from torch.utils.tensorboard import SummaryWriter

dummy_input=torch.randn(1,10)

writer = SummaryWriter(log_dir='./logs/exp2')
writer.add_graph(model, dummy_input)
writer.close()
# graphsで見ることができる

/Users/uedataiga/development/modellib/modellib_pytorch_env/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


## torchviz
- fc1とかまで細かく表示してくるからやや見にくい
- `cannot import name 'make_dot' from 'torchviz'`とかいう謎エラー起きた。最初は実行できていたので本当に意味がわからない

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F



INPUT_SIZE = 10

class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(INPUT_SIZE, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.softmax(x)

        return x

In [5]:

from torchviz import make_dot

INPUT_SIZE = 28*28

model = NeuralNet()
data = torch.randn(1, INPUT_SIZE)

y = model(data)

image = make_dot(y, params=dict(model.named_parameters()))
image.format = "png"
image.render("NeuralNet")

ImportError: cannot import name 'make_dot' from 'torchviz' (/Users/uedataiga/development/modellib/modellib_pytorch_env/lib/python3.7/site-packages/torchviz/__init__.py)

## torchsummary
- torchinfoとほぼ同じ、なんでこれがあるのかわからん

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision import datasets, transforms

# 追加============================
import os
from torchsummary import summary
# ===============================

from datetime import datetime

print(torch.__version__) # 1.5.0

# colabでgoogle driveをマウントしてない場合のパス
root="data/"

# dataの変換方法を定義
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# dataをダウンロード
train_set = datasets.MNIST(root=root, train=True, transform=trans, download=True)
test_set = datasets.MNIST(root=root, train=False, transform=trans, download=True)

# cpuかgpuか
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# dataloaderを定義
train_loader = DataLoader(train_set, batch_size=100, shuffle=True)
test_loader = DataLoader(test_set, batch_size=100, shuffle=False)

# Networkを定義
class MLPNet (nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(1 * 28 * 28, 512)
        self.fc2 =nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 10)
        self.dropout1=nn.Dropout2d(0.2)
        self.dropout2=nn.Dropout2d(0.2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        return F.relu(self.fc3(x))

net = MLPNet().to(device)

# torchsummaryを使った可視化
summary(net, input_size=(1,1 * 28 * 28))

1.1%

1.10.2


100.0%


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw


102.8%
4.1%


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



100.0%
112.7%

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 1, 512]         401,920
         Dropout2d-2               [-1, 1, 512]               0
            Linear-3               [-1, 1, 512]         262,656
         Dropout2d-4               [-1, 1, 512]               0
            Linear-5                [-1, 1, 10]           5,130
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 2.55
Estimated Total Size (MB): 2.57
----------------------------------------------------------------
